In [120]:
import pandas as pd

In [121]:
cu = (
    pd
    .read_csv(
        f'./assets/customers.csv',
        dtype={'home_phone_1': 'object', 'home_phone_2': 'object',
               'work_phone_1': 'object', 'work_phone_2': 'object',
               'mobile_1': 'object', 'mobile_2': 'object',
               'others_1': 'object', 'others_2': 'object'},
        usecols=['id', 'last_name', 'first_name', 'home_phone_1', 'home_phone_2', 'work_phone_1', 'work_phone_2', 'mobile_1', 'mobile_2', 'others_1', 'others_2', 'is_active']
    )
    .where(lambda x: (x['is_active'] == 1) & (x['is_active'].notna()))
    .dropna(how='all')
    .drop(columns=['is_active'])
)

cu['phone'] = cu['home_phone_1'].fillna(cu['home_phone_2']).fillna(cu['work_phone_1']).fillna(cu['work_phone_2']).fillna(cu['mobile_1']).fillna(cu['mobile_2']).fillna(cu['others_1']).fillna(cu['others_2']).astype(str)
cu = cu.drop(columns=['home_phone_1', 'home_phone_2', 'work_phone_1', 'work_phone_2', 'mobile_1', 'mobile_2', 'others_1', 'others_2'])
cu = cu.dropna(how='any')
cu['id'] = cu['id'].astype('int64')
cu.head()

,id,first_name,last_name,phone
0,1,Juan,Dela Cruz,1111111
3,4,Joel,Blanco,12345678
4,5,Juan,Dela Cruz,12345678
5,6,Michael,Bryant,12345678
6,7,Juan,Delario,12345678


In [122]:
ad = (
    pd
    .read_csv(
        f'./assets/addresses.csv',
        # parse_dates=['CreateDate', 'DeleteDate'],
        # date_format='%Y-%m-%d %H:%M:%S',
        # dtype={'id': 'int64', 'customer_id': 'int64'},
        usecols=['customer_id', 'address']
    )
    .dropna(how='any')
)

ad.head()

,customer_id,address
0,1,Capitol commons
1,2,"1213 Cabrera Road, San Roque"
2,3,block 32 lot 11 beverly hills 1st avenue
3,4,"no.21, Rosario STREET, ABA homes SUBDIVISION,..."
4,5,"16 brent wood, pines city executive village,sa..."


In [123]:
cu_add = cu.merge(ad,left_on='id',right_on='customer_id',how='left')
cu_add.head()

,id,first_name,last_name,phone,customer_id,address
0,1,Juan,Dela Cruz,1111111,1.0,Capitol commons
1,1,Juan,Dela Cruz,1111111,1.0,73 A.APOLINARIO ST COR GEN LACUA STREET\nBRGY ...
2,1,Juan,Dela Cruz,1111111,1.0,73 A.APOLINARIO ST COR GEN LACUA STREET\nBRGY ...
3,1,Juan,Dela Cruz,1111111,1.0,"13a basa st., roosevelt san francisco del mont..."
4,4,Joel,Blanco,12345678,4.0,"no.21, Rosario STREET, ABA homes SUBDIVISION,..."


In [124]:
ord = (
    pd.read_csv(
        f'./assets/orders.csv',
        parse_dates=['order_date'],
        date_format='%Y-%m-%d %H:%M:%S',
        dtype={'store_code': 'category'},
        usecols=['id', 'customer_id', 'order_date', 'store_code']
    )
)

ord.head()

,id,order_date,customer_id,store_code
0,1,2018-08-30 16:27:53,5,ANT
1,2,2018-08-30 16:30:17,6,ANT
2,3,2018-08-30 16:31:29,7,ANT
3,4,2018-08-31 07:41:10,7,ANT
4,5,2018-08-31 07:41:45,6,ANT


In [125]:
customer_orders = ord.merge(cu_add, left_on='customer_id', right_on='id', how='left')
customer_orders = customer_orders.rename(columns={'id_x': 'order_id', 'id_y': 'customer_id'}).drop(columns=['customer_id_y', 'customer_id_x'])
# customer_orders['customer_id'] = customer_orders['customer_id'].astype('int64')
customer_orders = customer_orders[customer_orders['customer_id'].notna()]
customer_orders['customer_id'] = customer_orders['customer_id'].astype('int64')
unique_orders = customer_orders.sort_values('order_date').drop_duplicates(subset=['customer_id'], keep='last')
pma = unique_orders[unique_orders['store_code'] == 'PMA']
psg = unique_orders[unique_orders['store_code'] == 'PSG']

pma.to_csv('./output/pma_unique_orders.csv', index=False)
psg.to_csv('./output/psg_unique_orders.csv', index=False)